# 6/18: API Data Collection - Sonoma wildfire time period: 10/2017 data

In [3]:
#import libraries
import io
import requests
import pandas as pd
import numpy as np

# Making url request

In [2]:
urlbase = "https://s3-us-west-1.amazonaws.com//files.airnowtech.org/airnow"
urlyear = "2017"
reporting_data_columns1 =  ['reporting_date', 'forecast_date', 'time', 'time_zone', 'unknown_1',
                       'unknown_binary_2', 'unknown_binary_3', 'reporting_area', 'state_code', 
                       'latitude', 'longitude', 'measurement_type', 'measurement_aqi', 
                       'aqi_interpretation', 'description', 'reporting_department']
air_data_total = None
air_data_reporting = None
month=6

In [5]:
#from factsheet <https://docs.airnowapi.org/docs/ReportingAreaFactSheet.pdf>
reporting_data_columns2 = ['issue date', 'valid date', 'valid time', 'time zone', 'record sequence', 'data type',
                          'primary', 'reporting area', 'state code', 'latitude', 'longitude', 'pollutant',
                          'AQI value', 'AQI category', 'action day', 'discussion', 'forecast source']

In [6]:
def unique_rowtag(row):
    return str(row['reporting area']) + str(np.int(row['latitude'])) + str(np.int(row['longitude']))

def get_reportingarea_data(day, month=10, year=2018):
    urldate = str(year) + str(month).zfill(2) +str(day).zfill(2)
    urlfilename = 'reportingarea.dat'
    s = requests.get(urlbase+'/'+urlyear+'/'+urldate+'/'+urlfilename).content
    air_data = pd.read_csv(io.StringIO(s.decode('utf-8')),sep='|', 
                          header=None, index_col = False, names=reporting_data_columns2)
    return air_data

# Load in data from Sonoma wildfire 2017 time period

In [7]:
#Sonoma wildfire: 10/8-10/31/2017
for i in np.arange(8, 32):
    reporting_data = get_reportingarea_data(i, month = 10, year = '2017')
    air_data_total = pd.concat([air_data_reporting, reporting_data], axis=0)
    
#air_data_total['unique_name'] = air_data_total.apply(unique_rowtag, axis=1)

In [8]:
test = get_reportingarea_data(8, month = 10, year = "2017")

test[test['data type'] == 0].head()
#print(get_reportingarea_data(9, month = 10, year = "2017")['valid date'].unique())

,issue date,valid date,valid time,time zone,record sequence,data type,primary,reporting area,state code,latitude,longitude,pollutant,AQI value,AQI category,action day,discussion,forecast source


In [9]:
air_data_total['valid date'].unique()
air_data_total.head()

,issue date,valid date,valid time,time zone,record sequence,data type,primary,reporting area,state code,latitude,longitude,pollutant,AQI value,AQI category,action day,discussion,forecast source
0,10/31/17,10/30/17,NaN,PDT,-1,Y,Y,Aberdeen,WA,46.9725,-123.8317,PM2.5,53.0,Moderate,No,NaN,Olympic Region Clean Air Agency
1,10/31/17,10/31/17,21:00,PDT,0,O,Y,Aberdeen,WA,46.9725,-123.8317,PM2.5,25.0,Good,No,NaN,Olympic Region Clean Air Agency
2,11/01/17,10/31/17,NaN,EAT,-1,Y,Y,Addis Ababa Central,,9.0585,38.7616,PM2.5,65.0,Moderate,No,NaN,U.S. Department of State Ethiopia - Addis Ababa
3,11/01/17,11/01/17,7:00,EAT,0,O,Y,Addis Ababa Central,,9.0585,38.7616,PM2.5,55.0,Moderate,No,NaN,U.S. Department of State Ethiopia - Addis Ababa
4,10/31/17,10/31/17,NaN,EDT,0,F,Y,Adirondacks Region,NY,43.9700,-74.1600,PM2.5,9.0,Good,No,NaN,New York Dept. of Environmental Conservation


In [12]:
air_data_total.to_csv('../Data/secondlook_all.csv')